In [3]:
%tensorflow_version 2.x

In [4]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla

--2021-06-09 14:03:16--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2021-06-09 14:03:16--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3514698bd72c1949938c6d036b.dl.dropboxusercontent.com/zip_download_get/AzfBuw9E6R5PPs_VbbrNl568loD0zDXnRJ6UplpGGv2RyabrDhZUR94qAVLi1cZgSzpbD0WU9mshCpeYQhRkmaogqSYpxYgTPuK30uCx5K7arQ# [following]
--2021-06-09 14:03:17--  https://uc3514698bd72c1949938c6d036b.dl.dropboxusercontent.com/zip_download_get/AzfBuw9E6R5PPs_VbbrNl568loD0zDXnRJ6UplpGGv2RyabrDhZUR94qAVLi1cZgSzpbD0WU9mshCpeYQhRkm

In [5]:
!unzip /content/AAAw1_JIzpuVvwteJCma0xMla



Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla
mapname:  conversion of  failed
 extracting: train.csv               


In [6]:
import pandas as pd
Quora = pd.read_csv(r"/content/train.csv")

In [7]:
Quora.head()


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [8]:
Quora.drop(["qid"],axis=1,inplace=True)

In [9]:
Quora

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [10]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip



--2021-06-09 14:05:52--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-06-09 14:05:53--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-06-09 14:05:53--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [11]:
!unzip /content/glove.42B.300d.zip


Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [12]:
!rm /content/glove.42B.300d.zip

In [13]:
import numpy as np

In [14]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [16]:
from sklearn.model_selection import train_test_split
Quora_train,Quora_test=train_test_split(Quora,test_size=0.2)

In [17]:
x_train=Quora_train['question_text']
y_train=Quora_train['target']
x_test=Quora_test['question_text']
y_test=Quora_test['target']

In [18]:


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model



In [19]:
from nltk import word_tokenize

In [20]:


import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [22]:
np.percentile(sent_lens,95)

31.0

In [23]:
max_len=31

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)



In [24]:
embedding_matrix=np.zeros((vocab_size+1,300))
words_not_available=0
for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
    else:
      words_not_available+=1



In [25]:
words_not_available

62161

In [26]:
vocab_size

196354

In [27]:
inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [28]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 31)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 31, 300)           58906500  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                70200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 58,977,221
Trainable params: 70,721
Non-trainable params: 58,906,500
____________________________________________

In [29]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)

In [31]:
class_weight_dict = dict(enumerate(cw))
class_weight_dict

{0: 0.5329986757885588, 1: 8.076061585073658}

In [32]:
model.fit(seq_train_matrix,y_train.values,validation_data=(seq_test_matrix,y_test.values),epochs=40,class_weight=class_weight_dict,
          batch_size=30000)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/40
35/35 [==============================] - 28s 580ms/step - loss: 0.5412 - accuracy: 0.7114 - val_loss: 0.3764 - val_accuracy: 0.8289
Epoch 2/40
35/35 [==============================] - 19s 550ms/step - loss: 0.3543 - accuracy: 0.8509 - val_loss: 0.3045 - val_accuracy: 0.8721
Epoch 3/40
35/35 [==============================] - 19s 552ms/step - loss: 0.3167 - accuracy: 0.8700 - val_loss: 0.3365 - val_accuracy: 0.8573
Epoch 4/40
35/35 [==============================] - 19s 553ms/step - loss: 0.3030 - accuracy: 0.8733 - val_loss: 0.3011 - val_accuracy: 0.8724
Epoch 5/40
35/35 [==============================] - 19s 557ms/step - loss: 0.2954 - accuracy: 0.8760 - val_loss: 0.2669 - val_accuracy: 0.8887
Epoch 6/40
35/35 [==============================] - 19s 558ms/step - loss: 0.2882 - accuracy: 0.8788 - val_loss: 0.2746 - val_accuracy: 0.8852
Epoc

In [33]:
p=model.predict(seq_test_matrix)

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.9587756078385348